In [42]:
import csv
import pathlib

import lxml.etree as etree

In [41]:
l10n_root = pathlib.Path('/Users/apollov/work/CDDA-new/CDDA/CDDA-loc')
l10n_csv = pathlib.Path('/Users/apollov/work/CDDA-new/CDDA/Translations.csv')

In [93]:
xliffs = list(l10n_root.glob('*/Localized Contents/*.xliff'))
xliffs

[PosixPath('/Users/apollov/work/CDDA-new/CDDA/CDDA-loc/zh.xcloc/Localized Contents/zh.xliff'),
 PosixPath('/Users/apollov/work/CDDA-new/CDDA/CDDA-loc/hu.xcloc/Localized Contents/hu.xliff'),
 PosixPath('/Users/apollov/work/CDDA-new/CDDA/CDDA-loc/ko.xcloc/Localized Contents/ko.xliff'),
 PosixPath('/Users/apollov/work/CDDA-new/CDDA/CDDA-loc/en.xcloc/Localized Contents/en.xliff'),
 PosixPath('/Users/apollov/work/CDDA-new/CDDA/CDDA-loc/de.xcloc/Localized Contents/de.xliff'),
 PosixPath('/Users/apollov/work/CDDA-new/CDDA/CDDA-loc/es.xcloc/Localized Contents/es.xliff')]

In [94]:
def csv_sanity_check():
    with open(l10n_csv) as f:
        reader = csv.reader(f)
        langs = next(reader)

        for record in reader:
            source = record[0]
            if (len(source) == 1) or (source[1] == ' '):
                sym = source[0]

                for lang, txn in zip(langs, record):
                    if txn[0] != sym:
                        print(f'Symbol {sym} differs for language {lang}: {txn}')
                    if (len(source) == 1) and (len(txn) != 1):
                        print(f'Trailing symbols for {sym} in {txn}')

csv_sanity_check()

In [95]:
def save_translation(xliff: pathlib.Path):
    with open(xliff) as f:
        tree = etree.parse(f)
    
    with open(l10n_csv) as f:
        translations_csv_reader = csv.reader(f)
        langs = next(translations_csv_reader)[1:]
        translation_mappings = {row[0]: dict(zip(langs, row[1:])) for row in translations_csv_reader}
    
    lang = xliff.name.split('.')[0]
    trans_units = findall(tree, 'trans-unit')
    
    for trans_unit in trans_units:
        source = findall(trans_unit, 'source')[0]
        
        try:
            translation = translation_mappings[source.text][lang]
        except KeyError:
            print(f'txn for {lang}/{source.text} not found. Skipping.')
            continue

        try:
            target = findall(trans_unit, 'target')[0]
        except IndexError:
            target = etree.SubElement(trans_unit, 'target')

        target.text = translation
        
    tree.write(str(xliff))


save_translation(xliffs[-2])

In [71]:
def findall(tree, tag):
    return tree.findall(f'.//{{urn:oasis:names:tc:xliff:document:1.2}}{tag}')


def get_targets_from_xliff(xliff):
    with open(xliff) as f:
        tree = etree.parse(f)
    
    targets = '\n'.join([x.text for x in findall(tree, 'target')])
    return targets

print(get_targets_from_xliff(xliffs[0]))

N Váltás a minimapon
1
U Rakja ki vagy ürítse ki az árnyékolt tárgyat
m Térkép megtekintése
Info
7
L Mozgás nézet kelet felé
0 Súgó megtekintése
: Közép nézet
Néz
4
D
R Olvassa el
Y Zónák kezelése
Leltár
V Sorolja fel az összes elemet a lejátszó körül
I
W
A Hordozható elem alkalmazása vagy használata
/ Fejlett készletgazdálkodás
v Morál megtekintése
{ Térképmemória váltása
Egyéb
6
3
9
TÉR
a Alkalmazás vagy használat elemet
& Kézműves termékek
_ Válassza ki a harcművészeti stílust
} Kapcsolja be a panel adminisztrátorát
+ Páncél / ruházat újrarétegezése
Harc
2
TÉR
P Üzenetnapló megtekintése
X Kukucskáljon a sarkok mögött
[ Mutációk megtekintése / aktiválása
= Cserélje le a betűjeleket
G Fogj valamit a közelben
K Mozgás északra
( Szerelje szét az elemeket
e Vizsgálja meg a közeli terepet
) Pontszámok megtekintése
c Csukja be az ajtót
| Várjon néhány percet
> Lépjen le a lépcsőn
D Dobja az elemet a szomszédos csempére
- Újrafogalmazza az utolsó receptet
TAB
" Mozgás mód menü
J Nézet mozga

In [ ]:
etree.dump(trans_unit)